In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
def dilated_convolution_block(inputs):
    """
    Apply dilated convolutions with different dilation rates
    to capture features at different scales
    """
    # First dilated convolution with rate 1
    conv1 = layers.Conv2D(32, kernel_size=3, dilation_rate=1, 
                         padding='same', activation='relu')(inputs)
    
    # Second dilated convolution with rate 2
    conv2 = layers.Conv2D(32, kernel_size=3, dilation_rate=2,
                         padding='same', activation='relu')(conv1)
    
    # Third dilated convolution with rate 4
    conv3 = layers.Conv2D(32, kernel_size=3, dilation_rate=4,
                         padding='same', activation='relu')(conv2)
    
    # Concatenate all dilated convolution outputs
    return layers.Concatenate()([conv1, conv2, conv3])

In [4]:
def multiscale_feature_learning(inputs):
    """
    Parallel convolutions with different kernel sizes for 
    multi-scale feature extraction
    """
    # Different kernel sizes for capturing different scales
    conv1 = layers.Conv2D(32, kernel_size=1, activation="relu", padding="same")(inputs)
    conv3 = layers.Conv2D(32, kernel_size=3, activation="relu", padding="same")(inputs)
    conv5 = layers.Conv2D(32, kernel_size=5, activation="relu", padding="same")(inputs)
    
    # Concatenate features from different scales
    return layers.Concatenate()([conv1, conv3, conv5])


In [5]:
def build_model(input_shape=(28, 28, 1), num_classes=10):
    # Input layer
    inputs = layers.Input(shape=input_shape)
    
    # 1. Dilated Convolution Block
    x = dilated_convolution_block(inputs)  # Output: (28, 28, 96) [32*3 channels]
    
    # 2. Multiscale Feature Learning
    multiscale_features = multiscale_feature_learning(x)  # Output: (28, 28, 96) [32*3 channels]
    
    # 3. Residual Connection
    x = layers.Add()([x, multiscale_features])
    
    # 4. Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Classification head
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    return models.Model(inputs, outputs)

In [6]:
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 28, 28,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 28, 28,    │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 28,    │      9,248 │ conv2d_1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 28, 28,    │          0 │ conv2d[0][0],     │
│ (Concatenate)       │ 96)               │            │ conv2d_1[0][0],   │
│                     │                   │            │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │      3,104 │ concatenate[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 28, 28,    │     27,680 │ concatenate[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 28, 28,    │     76,832 │ concatenate[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 28, 28,    │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 96)               │            │ conv2d_4[0][0],   │
│                     │                   │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 28, 28,    │          0 │ concatenate[0][0… │
│                     │ 96)               │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 96)        │          0 │ add[0][0]         │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     12,416 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10)        │      1,290 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 140,138 (547.41 KB)

 Trainable params: 140,138 (547.41 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=2,
    validation_split=0.2,
)

Epoch 1/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 96s 126ms/step - accuracy: 0.4661 - loss: 1.4327 - val_accuracy: 0.9333 - val_loss: 0.2365
Epoch 2/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 94s 126ms/step - accuracy: 0.9147 - loss: 0.2939 - val_accuracy: 0.9477 - val_loss: 0.1742


In [10]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9406 - loss: 0.1826

Test accuracy: 0.9510
